<a href="https://colab.research.google.com/github/svetaU/Attention/blob/main/SA_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import torch
try:
  import einops
except ModuleNotFoundError: 
  !pip install --quiet einops
from einops import rearrange
from torch import nn
#try:
#    import pytorch_lightning as pl
#except ModuleNotFoundError: 
#    !pip install --quiet pytorch-lightning>=1.5
#    import pytorch_lightning as pl